In [1]:
! pip3 install re

ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re


In [2]:
! pip3 install snownlp

In [3]:
! pip3 install jieba

In [4]:
! python match.py

python: can't open file 'match.py': [Errno 2] No such file or directory


## =====Parameter Definition=====

In [1]:
import os

# ----- ptt_board to crawl
ptt_board = 'Tech_Job'
data_root = os.path.abspath(os.getcwd())+ '/data/'
raw_data_path = data_root+'ptt_'+ptt_board+'/raw/'

# ----- data with basic processing
# processed_data_path = data_root+'processed_sex/'
processed_data_path = data_root+'ptt_'+ptt_board+'/processed/'

# structurized csv files
csv_path = processed_data_path + ptt_board + '.csv'
# # Data Cleaning and Tokenizing
# tok_csv_path = processed_data_path + ptt_board + '_tok' + '.csv'
w2v_csv_path = processed_data_path + ptt_board + '_w2v' + '.csv'

model_path = './models/' + ptt_board + '/'
model_name = ptt_board + 'w2v_model_all.md'

if not os.path.isdir(model_path):
    os.makedirs(model_path)
    print("made folder:", model_path)

In [ ]:
# print(os.path.abspath(os.getcwd()))

## =====Run Crawler & ETL===== 

 ## Data Collection

In [ ]:
# # Crawl data from ptt according to given board. 
# # you can also edit Crawler.py and run it in the shell 

# from Crawler import PttCrawler

# crawler = PttCrawler()
# crawler.crawl(board=ptt_board, start=2, end=5)

## Data Preprocessing

In [ ]:
# ##--- process_raw_data

# from filter import ArticleFilter

# Filter = ArticleFilter(data_root, processed_data_path)

# Filter.process_raw_data(raw_data_path, is_dir=True,to_one_file=True,one_file_name="CorpusPatch2.json")
# Filter.merge_coprus()

In [ ]:
##--- load processed data for further cleaning

# from filter import ArticleFilter

# Filter = ArticleFilter(data_root, processed_data_path)

# Filter.load_processed_corpus(processed_data_path)
# # Filter.print_titles()
# # Filter.print_response()
# # Filter.print_user_info() TODO?


In [ ]:
# print(len(Filter.order_response))
# print(Filter.order_titles[0])
# print(Filter.order_content[0])
# print(Filter.order_response[0])
# print(Filter.order_votes[0])
# print(Filter.order_users[0])
# print(Filter.order_users[0])

In [ ]:
# import pandas as pd

# pd_corpus = pd.DataFrame(data={'title': Filter.order_titles, 'content': Filter.order_content, 'response':Filter.order_response,
#                               'vote': Filter.order_votes, 'user': Filter.order_users, 'date': Filter.order_date,})

# pd_corpus.head()

In [ ]:
# # pd_corpus.title = pd_corpus.title.str.encode('utf-8')
# # pd_corpus.content = pd_corpus.content.str.encode('utf-8')
# # pd_corpus.response = pd_corpus.response.str.encode('utf-8')

# pd_corpus['Re'] = False
# pd_corpus['Fw'] = False

# pd_corpus.loc[pd_corpus.title.str.contains('Re'), 'Re'] = True
# pd_corpus.loc[pd_corpus.title.str.contains('Fw'), 'Fw'] = True

# pd_corpus.head()

In [ ]:
# pd_corpus.to_csv(csv_path)

## =====Word2Vec=====

## Load data

In [6]:
from snownlp import SnowNLP
import jieba
import os 
import pandas as pd


pd_corpus = pd.read_csv(csv_path)

print(pd_corpus.head())


FileNotFoundError: [Errno 2] No such file or directory: '/home/jovyan/work/ten/data/ptt_Tech_Job/processed/Tech_Job.csv'

## Data Cleaning and Tokenizing

In [8]:
'''
Note: expensive(slow)

'''


import re



def re_filter(x, seed):
    '''
    '''
    
    
#     print(type(x))
    
    for s in re.findall(seed, x):
        if s == "\n":
            x = x.replace(s, " ")
        else:
            x = x.replace(s, "")
    return x

# pd_corpus.title = pd_corpus.title.astype('str')
pd_corpus.content = pd_corpus.content.astype('str')
# pd_corpus.response = pd_corpus.response.astype('str')
 

pd_corpus['content'] = pd_corpus.content.apply(re_filter, args=("[0-9]{2}/[0-9]{2} [0-9]{2}:[0-9]{2}",))
pd_corpus['content'] = pd_corpus.content.apply(re_filter, args=("時間[\w]{3} [\w]{3} [\w]{2} [\w]{2}:[\w]{2}:[\w]{2} [\w]{4}",))
pd_corpus['content'] = pd_corpus.content.apply(re_filter, args=("時間[\w]{3} [\w]{3} [\w]{2} [\w]{2} [\w]{4}",))
pd_corpus['content'] = pd_corpus.content.apply(re_filter, args=("作者[a-z]*[A-Z]*[0-9]* .*標題",))
pd_corpus['content'] = pd_corpus.content.apply(re_filter, args=("[\w]*:",))
pd_corpus['content'] = pd_corpus.content.apply(re_filter, args=("發信站: ",))
pd_corpus['content'] = pd_corpus.content.apply(re_filter, args=("[0-9]+\.[0-9]+\.[0-9]+\.[0-9]+",))
pd_corpus['content'] = pd_corpus.content.apply(re_filter, args=("批踢踢實業坊\(ptt.cc\)|PTT|PTT2|PTT網頁板",))
pd_corpus['content'] = pd_corpus.content.apply(re_filter, args=("文章網址",))
pd_corpus['content'] = pd_corpus.content.apply(re_filter, args=(".*網誌.*",))
pd_corpus['content'] = pd_corpus.content.apply(re_filter, args=("轉錄至看板|轉錄至某隱形看板",))
pd_corpus['content'] = pd_corpus.content.apply(re_filter, args=("※|◆|--|~|→|:|\n|/|\.|=|!",))
pd_corpus['content'] = pd_corpus.content.apply(re_filter, args=("  ",)) 

pd_corpus.head()

NameError: name 'pd_corpus' is not defined

In [ ]:
'''
Note: very expensive(slow) when use_paddle=True

'''



def to_sentences(x):
    snow = SnowNLP(x)
    S = ""
    for s in snow.sentences:
        S += s
        S += ","
        
    return S

def to_words(x, stop_words):
    words = jieba.cut(x, cut_all=False, use_paddle=False)
#     words = jieba.cut(x, cut_all=False, use_paddle=True)
    W = []
    for w in words:
        if w not in stop_words and w != '\n' and w != ' ':
            W.append(w)
        
    return W

def loadStopWords(path):
    '''
    '''
    
    stopwords = []
    
    with open(path, 'r', encoding='utf-8') as sw:
        for word in sw:
            stopwords.append(word.strip('\n'))
    
    return stopwords

def loadFilterdWord(path):
    '''
    '''
    
    filteredWords = []
    
    with open(path, 'r', encoding='utf-8') as sw:
        for word in sw:
            filteredWords.append(word.strip('\n'))
            
    return filteredWords
            


In [ ]:
stopwords = loadStopWords(path="data/stopwords/stop_words.csv")
print(stopwords)

pd_corpus = pd_corpus[pd_corpus.content!=""]

# Tokenize the contests (documents)
pd_corpus['sentences'] = pd_corpus.content.apply(to_sentences)
pd_corpus['words'] = pd_corpus.sentences.apply(to_words, args=(stopwords,))

print(pd_corpus['sentences'][1])
print(pd_corpus['words'][1])


In [ ]:
pd_corpus.to_csv(w2v_csv_path)

In [7]:
!pip install gensim

## ===== Train & Save W2V models =====

In [2]:
from gensim.models.word2vec import Word2Vec
import pandas as pd
import ast 



pd_corpus = pd.read_csv(w2v_csv_path)

# Select Only unique documents
corpus = pd.DataFrame(pd_corpus.words.sample(frac=1).unique(),
                   columns=['words'])

# type converting
corpus.words = corpus.words.apply(lambda x: ast.literal_eval(x))
# corpus.words = corpus.words.apply(lambda x: x.strip('][').split(', '))

corpus.head()

,words
0,"[好, 小弟, 一家, 八, 吋, foundry, 做, staff, RD, 去, 聽,..."
1,"[版上, 先進, 請問, 有人, 新加坡, GF, 做, 製程, 整合, 小弟, 想, 請教..."
2,"[沒, 市場, 電量, 估算, 電池, 材料, 特性, 鋰, 鈷, 做爛, chip, 做,..."
3,"[一家, 公司, 談到, 薪資, 時說, 全薪, 需, 抽, 5%, 說, 每半年, 達成,..."
4,"[先寫, 兩份, 筆試, 考卷, HR, 去, 請, 主管, 進來, 面試, 借, 幾分鐘,..."


In [6]:
import os


model_path_cbow = model_path + model_name + '.' + 'cbow'
model_path_sg = model_path + model_name + '.' + 'sg'

model = Word2Vec(corpus.words)
model.save(model_path_cbow)

model = Word2Vec(corpus.words, sg=1)
model.save(model_path_sg)

# W2V_model_250d_cbow = Word2Vec(corpus_all, size=250, iter=13)
# W2V_model_250d_sg = Word2Vec(corpus_all, size=250, iter=13, sg=1)

In [7]:
model_path_cbow = model_path + model_name + '.' + 'cbow'
model_path_sg = model_path + model_name + '.' + 'sg'

word2vec_sg = Word2Vec.load(model_path_sg)
word2vec_cbow = Word2Vec.load(model_path_cbow)

In [8]:
word2vec_sg.wv.vocab

{'好': <gensim.models.keyedvectors.Vocab at 0x7fb983c61e20>,
 '小弟': <gensim.models.keyedvectors.Vocab at 0x7fb921f98130>,
 '一家': <gensim.models.keyedvectors.Vocab at 0x7fb921f980d0>,
 '八': <gensim.models.keyedvectors.Vocab at 0x7fb921f98310>,
 '吋': <gensim.models.keyedvectors.Vocab at 0x7fb921f981c0>,
 'foundry': <gensim.models.keyedvectors.Vocab at 0x7fb921ee31c0>,
 '做': <gensim.models.keyedvectors.Vocab at 0x7fb921ee3220>,
 'staff': <gensim.models.keyedvectors.Vocab at 0x7fb921ee32e0>,
 'RD': <gensim.models.keyedvectors.Vocab at 0x7fb921ee3340>,
 '去': <gensim.models.keyedvectors.Vocab at 0x7fb921ee33a0>,
 '聽': <gensim.models.keyedvectors.Vocab at 0x7fb921ee3400>,
 '自強': <gensim.models.keyedvectors.Vocab at 0x7fb921ee37c0>,
 '基金會': <gensim.models.keyedvectors.Vocab at 0x7fb921ee3040>,
 'PMP': <gensim.models.keyedvectors.Vocab at 0x7fb980c3cf40>,
 '課程': <gensim.models.keyedvectors.Vocab at 0x7fb980c3c760>,
 '說明會': <gensim.models.keyedvectors.Vocab at 0x7fb980c3ce80>,
 '動心': <gensim.mode

In [9]:
word2vec_sg.wv.similarity('感恩', '社會')

0.28299925

In [10]:
word2vec_sg.wv.most_similar('感恩', topn=10)

[('感激不盡', 0.8924201726913452),
 ('非常感謝', 0.8656332492828369),
 ('謝謝', 0.8596808910369873),
 ('萬分感激', 0.8439638614654541),
 ('謝謝你們', 0.8293709754943848),
 ('非常感激', 0.8286060094833374),
 ('萬分', 0.8228143453598022),
 ('跪謝', 0.806949257850647),
 ('好心人', 0.8058849573135376),
 ('不吝', 0.8047784566879272)]

In [11]:
word2vec_sg.wv.most_similar('公司', topn=10)

[('算大', 0.7242927551269531),
 ('工司', 0.722392737865448),
 ('頃向', 0.7119271755218506),
 ('歐商', 0.6981695890426636),
 ('不大好', 0.6962050795555115),
 ('公司員工', 0.695170521736145),
 ('二者', 0.6950112581253052),
 ('二家', 0.6944388151168823),
 ('輝瑞', 0.6932104825973511),
 ('比較完善', 0.6905535459518433)]

In [12]:
word2vec_sg.wv.most_similar('工作', topn=10)

[('工做', 0.732524037361145),
 ('想換個', 0.7133961319923401),
 ('憂慮', 0.6917105317115784),
 ('相仿', 0.6841260194778442),
 ('問裡面', 0.683264970779419),
 ('偏愛', 0.681769609451294),
 ('有想換', 0.6779323816299438),
 ('工司', 0.6766245365142822),
 ('得來不易', 0.6733204126358032),
 ('敝司', 0.6712121367454529)]

In [13]:
word2vec_sg.wv.most_similar('面試', topn=10)

[('面談', 0.8176228404045105),
 ('我面', 0.7767248153686523),
 ('二面', 0.7754706740379333),
 ('複試', 0.7630352973937988),
 ('有面', 0.7629225850105286),
 ('要面', 0.7543685436248779),
 ('兩關', 0.7541449666023254),
 ('面過', 0.7517653107643127),
 ('三月中', 0.7486779689788818),
 ('初試', 0.7475554943084717)]

In [14]:
# model.wv.get_vector('感恩')

In [15]:
word2vec_sg.most_similar(positive=['聯發科', '台積電'], negative=['社會'], topn=1)

<ipython-input-15-487797f23b8d>:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  word2vec_sg.most_similar(positive=['聯發科', '台積電'], negative=['社會'], topn=1)


[('發科', 0.6931043863296509)]

In [17]:
import pandas as pd


def most_similar(w2v_model, words, topn=10):
    '''
    '''
    
    similar_df = pd.DataFrame()
    for word in words:
        try:
            similar_words = pd.DataFrame(w2v_model.wv.most_similar(word, topn=topn), columns=[word, 'cos'])
            similar_df = pd.concat([similar_df, similar_words], axis=1)
        except:
            print(word, "not found in Word2Vec model!")
    return similar_df

def get_embedding(x, model):
    '''
    '''
    
    if x in model.wv.vocab:
        return model.wv.get_vector(x)
    else:
        return []
    

def get_similar_keywords(x, model, topn=10, ):
    '''
    '''
    
    if x in model.wv.vocab:
        similar_words = pd.DataFrame(model.wv.most_similar(x, topn=topn), columns=['word', 'cos'])
        return similar_words['word'].values
#         return model.wv.most_similar(x, topn=topn)
    else:
        return []
  

In [19]:
# Select Only unique documents
corpus = pd.DataFrame(pd_corpus.words.sample(frac=1).unique(),
                   columns=['words'])[:20000]

# type converting
corpus.words = corpus.words.apply(lambda x: ast.literal_eval(x))
corpus.words = corpus.words.apply(lambda x: ', '.join(x))

corpus

,words
0,"好, 有幸, 收到, 兩間, 公司, Offer, 義隆和奕力, 兩間, 年薪, 差不多, ..."
1,"小弟, 月, 中, 台積, 面試, 14, 廠, 校園, 徵才, 當天, 面試, 兩位, 主..."
2,"就讀, 私立, 大學, 電機系, 研替, 面試, 機會, 不, 運氣, 還, 算, 不錯, ..."
3,"小蛇, 國立, 縣市, 名, 企管, MBA, 工作, 經驗, 年, 快, 半, 都, 外派..."
4,"版上, 好, 小弟, 三月初, 去, 麥, 寮, 參加, 面試, 三月底, 接到, 通知, ..."
...,...
19995,"好, 板, 上, 獲取, 不少, 資訊, 基於, 回饋, 面試, 路, 分享, 面試, 過的..."
19996,"科風, 公司, 主動, 打電話, 詢問, 面試, 意願, 工作, 性質, PV, Invet..."
19997,"公開, 資訊, 觀測站, 列出, 全體, 員工, 人數, 非, 擔任, 主管, 職, 全時,..."
19998,"說真的, 每家, 公司, 老闆, 排, 兩三個, 人, 差不多, 時間, 面試, 時, 都,..."


In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
# compute tf_idf
vectorizer = TfidfVectorizer()
feat_arr = vectorizer.fit_transform(corpus['words'])
tokens = vectorizer.get_feature_names()
word_idf = pd.DataFrame({'word': vectorizer.get_feature_names(),
                        'tf_idf': feat_arr.toarray().sum(axis=0)})
word_idf

,word,tf_idf
0,00,52.269448
1,000,13.075731
2,0000,21.643688
3,000000,0.719007
4,00000000,2.157986
...,...,...
91659,龜毛,0.590496
91660,龜縮,0.200012
91661,龜苓膏,0.298888
91662,龜速,0.390296


In [22]:
word_idf.sort_values(by='tf_idf', ascending=False)[:10]

,word,tf_idf
27047,公司,783.876497
89152,面試,680.199821
42541,工作,631.509622
78679,請問,462.722024
42685,工程師,446.430293
41620,小弟,438.716675
79020,謝謝,344.900973
20326,主管,327.900262
29023,前輩,314.036120
85433,部門,264.125237


In [44]:
 most_similar(word2vec_sg,["公司","面試","工作","請問","工程師","小弟","謝謝","主管","前輩","部門"])

,公司,cos,面試,cos,工作,cos,請問,cos,工程師,cos,小弟,cos,謝謝,cos,主管,cos,前輩,cos,部門,cos
0,算大,0.724293,面談,0.817623,工做,0.732524,請教,0.910210,副工,0.769845,小妹,0.883101,非常感謝,0.899560,主官,0.787517,板友們,0.852324,TEAM,0.744149
1,工司,0.722393,我面,0.776725,想換個,0.713396,想問,0.828586,技術部,0.754913,小魯,0.876848,感激不盡,0.887593,部門經理,0.781213,先進,0.843273,單位,0.743554
2,頃向,0.711927,二面,0.775471,憂慮,0.691711,問版上,0.800019,二部,0.753737,小女子,0.804205,感恩,0.859681,面試官,0.773215,版大,0.840011,BU,0.743008
3,歐商,0.698170,複試,0.763035,相仿,0.684126,問問看,0.799364,師四,0.750204,小魯弟,0.803537,感謝,0.859173,甚歡,0.768435,版友,0.836061,研發部門,0.721015
4,不大好,0.696205,有面,0.762923,問裡面,0.683265,問問,0.791729,開發部,0.750039,本人,0.794357,謝過,0.840164,相談,0.758513,版大們,0.835083,研發部,0.704144
5,公司員工,0.695171,要面,0.754369,偏愛,0.681770,問下,0.763450,服務部,0.748478,不才,0.772758,跪謝,0.822233,聊完,0.754436,先輩,0.832421,DTP,0.693853
6,二者,0.695011,兩關,0.754145,有想換,0.677932,我想問,0.749081,CELL,0.747777,魯弟,0.772632,非常感激,0.803013,人事主管,0.751603,板友,0.829547,工作部門,0.687989
7,二家,0.694439,面過,0.751765,工司,0.676625,問板,0.743746,二處,0.745054,本魯為,0.769784,謝謝你們,0.801152,健談,0.748570,大德,0.806181,BSID,0.687768
8,輝瑞,0.693210,三月中,0.748678,得來不易,0.673320,詢問,0.734227,cim,0.744947,魯妹,0.764004,萬分感激,0.793135,人資,0.748206,板大們,0.805895,BU1,0.686482
9,比較完善,0.690554,初試,0.747555,敝司,0.671212,尋問,0.731513,乙職,0.744506,好小魯,0.761847,謝謝您,0.790284,開門見山,0.745722,過或有,0.802125,team,0.684831


In [24]:
get_embedding("離職",word2vec_sg)

array([-0.17543308, -0.26401895, -0.03488924, -0.18672697,  0.53166366,
        0.14535958,  0.36037108, -0.5113933 ,  0.6142428 ,  0.41027558,
        0.08834065, -0.4067236 , -0.6542274 ,  0.18254809, -0.17528477,
        0.26476288, -0.14331867, -0.13963751, -0.11262336,  0.28865963,
       -0.27548075, -0.4358036 , -0.4287073 ,  0.34917897, -0.21506785,
       -0.31551448,  0.30338153,  0.08319834,  0.25619635,  0.71248037,
       -0.7249309 , -0.13913624,  0.17792332, -0.24222392, -0.64778733,
       -0.44729334, -0.08024067,  0.00416889, -0.50265944,  0.27225122,
        0.3642015 ,  0.28248766, -0.7753535 ,  0.18681504, -0.44126904,
        0.701518  , -0.44754788, -0.14011948, -0.01657888,  0.08754181,
        0.24301562, -0.13125771, -0.289087  ,  0.09544569,  0.37328947,
       -0.02584445,  0.5075329 , -0.017167  , -0.41699016,  0.2273396 ,
        0.00824092,  0.6524805 ,  0.02540658,  0.3405729 ,  0.61855686,
       -0.01877912, -0.47046348, -0.34673676, -0.3353198 , -0.16

In [30]:
get_similar_keywords("單身", word2vec_cbow, topn=10, )

array(['要養', '買房', '老婆', '太太', '父母', '買車', '夫妻', '小孩', '娶', '房貸'],
      dtype=object)

In [60]:
get_similar_keywords("單身",word2vec_sg, topn=10, )

array(['未婚', '雙親', '有車', '有房', '後援', '單薪', '小家庭', '沒房', '另一半', '夫妻'],
      dtype=object)